In [175]:
from dotenv import load_dotenv
import urllib.parse as urlparse
import requests
import json
import os
from datetime import datetime

class Untappd:

    def __init__(self, clientId = "", clientSecret = ""):
        load_dotenv(dotenv_path="../.env")
        self._clientId = os.getenv("CLIENT_ID_ALT")
        self._clientSecret = os.getenv("CLIENT_SECRET_ALT")
        self._baseUrl = "https://api.untappd.com/v4/"


    def keys(self):
        return {
            "client_id": self._clientId,
            "client_secret": self._clientSecret
        }

    def findBrewery(self, brewery="", writeFile=False):
        methodUrl = self._baseUrl + "search/brewery"
        params = {
            "q": brewery
        }
        requestUrl = methodUrl + "?" + urlparse.urlencode({**params, **self.keys()})
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        breweries = [] 
        
        if response.status_code == 200:
            content = response.json()
            
            
            if content["response"]["found"] > 0:
                breweries = content["response"]["brewery"]["items"]
        
        return breweries, counter, response
    
    def breweryInfo(self, breweryId="", writeFile=False):
        methodUrl = self._baseUrl + "brewery/info/"
        requestUrl = methodUrl + str(breweryId) + "?" + urlparse.urlencode(self.keys())
        response = requests.get(requestUrl)
        
        headers = response.headers
        counter = int(headers["X-Ratelimit-Remaining"])
        
        if response.status_code == 200:
            content = response.json()

        return content["response"], counter, response



### Create a brewery dataset

In [55]:
import numpy as np
import pandas as pd
import pickle

try:
    with open ("../data/all_breweries.pkl", "rb") as fp:
        all_breweries = pickle.load(fp)
except:
    print( "No file existed, creating one!" )
    breweries = pd.read_csv("../data/Texas Breweries.csv")
    brewpubs = pd.read_csv("../data/Texas Brewpubs.csv")
    brewery_list = list(breweries.T.to_dict().values())
    all_breweries = brewery_list + list( brewpubs.T.to_dict().values() )

    
    
    with open('../data/all_breweries.pkl', 'wb') as fp:
        pickle.dump(all_breweries, fp)
        
display( all_breweries[0:2] )

[{'Name': '111 Brewing',
  'Street Address': 111,
  'Road': 'S FANNIN AVENUE',
  'City': 'DENISON',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '4/10/2019',
  'Status': 'In the works'},
 {'Name': '2 Docs Brewing Co',
  'Street Address': 502,
  'Road': 'TEXAS AVENUE',
  'City': 'LUBBOCK',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '10/5/2018',
  'Status': 'Open'}]

Due to untappd's 100 requests an hour, i'll need to run this over a couple of hours.  Thus saving results for future reference.

In [3]:
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

untappd = Untappd()
 
for brewery in all_breweries:
    if "untappd" not in brewery:
        brewery_result, counter, response = untappd.findBrewery(brewery["Name"])
        
        if len( brewery_result ) > 0:
            brewery["untappd"] = brewery_result
        else:
            brewery["untappd"] = [{"no_results":True}]

        clear_output(wait=True)
        display(counter)
        if counter == 0:
            break
        
#         break
#Save results
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)

print( "sample output" )
print( all_breweries[0] )

82

sample output
{'Name': '111 Brewing', 'Street Address': 111, 'Road': 'S FANNIN AVENUE', 'City': 'DENISON', 'State': 'TX', 'Brewery License': 'Brewery', 'License Date': '4/10/2019', 'Status': 'In the works', 'untappd': [{'brewery': {'brewery_id': 66888, 'beer_count': 65, 'brewery_name': 'Milepost 111 Brewing Company', 'brewery_slug': 'milepost-111-brewing-company', 'brewery_page_url': '/milepostbeer', 'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg', 'country_name': 'United States', 'location': {'brewery_city': 'Cashmere', 'brewery_state': 'WA', 'lat': 47.5244, 'lng': -120.47}}}]}


In [148]:
import pickle
import pandas as pd
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)
 
# Normalized the results
brewery_parsed = []
i = 0
for brewery in all_breweries:
    clear_output(wait=True)
    display(i)
    i+=1
    for entries in brewery["untappd"]:
        if "no_results" not in entries:
            brewery_parsed.append({
                "Name":brewery["Name"],
                "Street Address":brewery["Street Address"],
                "Road":brewery["Road"],
                "City":brewery["City"],
                "State":brewery["State"],
                "Brewery License":brewery["Brewery License"],
                "License Date":brewery["License Date"],
                "Status":brewery["Status"],
                "search_count":len(brewery["untappd"]),
                "brewery_id":entries["brewery"]["brewery_id"],
                "brewery_name":entries["brewery"]["brewery_name"],
                "beer_count":entries["brewery"]["beer_count"],
                "brewery_city":entries["brewery"]["location"]["brewery_city"],
                "brewery_state":entries["brewery"]["location"]["brewery_state"],
                "brewery_lat":entries["brewery"]["location"]["lat"],
                "brewery_lng":entries["brewery"]["location"]["lng"]
            })
        

# Remove results where not in texas
brewery_parsed = [ brewery for brewery in brewery_parsed if "TX" in brewery["brewery_state"]]
# Must have 10 beers!
brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["beer_count"] > 9 ]

#remove duplicates
brewery_count = pd.DataFrame(brewery_parsed).groupby(["Name"])["Status"].count()
dups = brewery_count.loc[ brewery_count > 1].reset_index()
dups = dups["Name"].to_list()
print("Duplicates to be removed", dups)
brewery_parsed = [ brewery for brewery in brewery_parsed if brewery["Name"] not in dups ]

#Add Brewery ID back in to primary data
for brewery in all_breweries:
    for b in brewery_parsed:
        if brewery["Name"] in b["Name"]:
            brewery["brewery_id"] = b["brewery_id"]
            
#Save results
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)
# #Convert to Pandas and save it
# # brewery_parsed = pd.DataFrame( brewery_parsed )
# # brewery_parsed.to_csv("../data/brewery_search.csv")
pd.options.display.max_rows = 999
display( pd.DataFrame( brewery_parsed ) )
pd.options.display.max_rows = 15

365

Duplicates to be removed ['Flix Brewhouse', 'Freetail Brewing Co.', 'Hops And Grain', 'Malai Kitchen', 'Pinthouse Pizza']


,Name,Street Address,Road,City,State,Brewery License,License Date,Status,search_count,brewery_id,brewery_name,beer_count,brewery_city,brewery_state,brewery_lat,brewery_lng
0,2 Docs Brewing Co,502,TEXAS AVENUE,LUBBOCK,TX,Brewery,10/5/2018,Open,1,420745,Two Docs Brewing Co.,22,Lubbock,TX,33.5909,-101.8460
1,3 Nations Brewing,2405,SQUIRE PLACE SUITE 200,FARMERS BRANCH,TX,Brewery,5/6/2015,Open,1,205645,3 Nations Brewing,43,Farmers Branch,TX,32.9310,-96.8983
2,512 Brewing Company,407,RADAM LANE 'F200',AUSTIN,TX,Brewery,5/20/2008,Open,1,1,(512) Brewing Company,156,Austin,TX,30.2236,-97.7697
3,8th Wonder Brewery,2202,DALLAS STREET,HOUSTON,TX,Brewery,12/29/2011,Open,1,45113,8th Wonder Brewery,157,Houston,TX,29.7492,-95.3558
4,Alamo Beer,415,BURNET STREET,SAN ANTONIO,TX,Brewery,9/29/2014,Open,1,43137,Alamo Beer Company,69,San Antonio,TX,29.4295,-98.4773
5,Armadillo Ale Works,221,S BELL AVENUE,DENTON,TX,Brewery,2/19/2015,Open,1,9316,Armadillo Ale Works™,28,Denton,TX,33.2126,-97.1281
6,Athens Brewing Company,101,E TYLER STREET,ATHENS,TX,Brewery,4/18/2016,Open,1,267312,Athens Brewing Company,35,Athens,TX,32.2054,-95.8538
7,Austin Beerworks,3009,INDUSTRIAL TERRACE SUITE 150,AUSTIN,TX,Brewery,12/30/2010,Open,1,11819,Austin Beerworks,266,Austin,TX,30.3798,-97.7302
8,Backstory Brewery,211,MAGNOLIA STREET,SULPHUR SPRINGS,TX,Brewery,10/31/2017,Open,1,393158,BackStory Brewery,40,Sulphur Springs,TX,33.1348,-95.6048
9,Bear King Brewing Company,207,AVE G,MARBLE FALLS,TX,Brewery,11/28/2018,Open,1,421863,Bear King Brewing Company,14,Marble Falls,TX,30.5703,-98.2749


In [176]:
import pickle
import numpy as np
import pandas as pd
import math
from IPython.display import clear_output

# Load data
with open ("../data/all_breweries.pkl", "rb") as fp:
    all_breweries = pickle.load(fp)

untappd = Untappd()

i = 0
for brewery in all_breweries:
    if "brewery_id" in brewery and "brewery_info" not in brewery:
        brewery_info, counter, response = untappd.breweryInfo(brewery["brewery_id"])

        brewery["brewery_info"] = brewery_info
        
        clear_output(wait=True)
        display(counter)
        if counter == 0:
            break
        
with open("../data/all_breweries.pkl", "wb") as fp:
    pickle.dump(all_breweries, fp)
    
print("Sample output")
display(all_breweries[1]["brewery_info"])

print("Completed on ", datetime.now())


0

Sample output


{'brewery': {'brewery_id': 420745,
  'brewery_name': 'Two Docs Brewing Co.',
  'brewery_slug': 'two-docs-brewing',
  'brewery_page_url': '/TwoDocsBrewingCo',
  'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-420745_c29c8.jpeg',
  'brewery_label_hd': '',
  'country_name': 'United States',
  'brewery_in_production': 0,
  'is_independent': 1,
  'claimed_status': {'is_claimed': True,
   'claimed_slug': 'TwoDocsBrewingCo',
   'follow_status': False,
   'follower_count': 72,
   'uid': 6376655,
   'mute_status': 'none'},
  'beer_count': 22,
  'contact': {'twitter': '',
   'facebook': 'https://www.facebook.com/twodocsbrewing',
   'instagram': 'twodocsbrewing',
   'url': 'https://twodocsbrewing.com/'},
  'brewery_type': 'Micro Brewery',
  'brewery_type_id': 2,
  'location': {'brewery_address': '502 Texas Ave',
   'brewery_city': 'Lubbock',
   'brewery_state': 'TX',
   'brewery_lat': 33.5909,
   'brewery_lng': -101.846},
  'rating': {'count': 1943, 'rating_score': 3.79

Completed on  2019-10-31 17:57:39.599953


In [54]:
from uszipcode import SearchEngine

for brewery in brewery_parsed:
    if abs(brewery["brewery_lat"]) > 0 and abs(brewery["brewery_lng"]) > 0:
        search = SearchEngine(simple_zipcode=False)
        result = search.by_coordinates(brewery["brewery_lat"],brewery["brewery_lng"], radius=30, returns=1)
        result = result[0].to_dict()
        display( {
            "brewery_city":brewery["brewery_city"],
            "major_city":result["major_city"],
            "state":result["state"],
            "zipcode":result["zipcode"]
        } )
    
    

{'brewery_city': 'Cashmere',
 'major_city': 'Cashmere',
 'state': 'WA',
 'zipcode': '98815'}

{'brewery_city': 'Lubbock',
 'major_city': 'Lubbock',
 'state': 'TX',
 'zipcode': '79401'}

{'brewery_city': 'Farmers Branch',
 'major_city': 'Dallas',
 'state': 'TX',
 'zipcode': '75234'}

{'brewery_city': 'Austin',
 'major_city': 'Austin',
 'state': 'TX',
 'zipcode': '78704'}

{'brewery_city': 'Houston',
 'major_city': 'Houston',
 'state': 'TX',
 'zipcode': '77003'}

...

{'brewery_city': 'Katy',
 'major_city': 'Katy',
 'state': 'TX',
 'zipcode': '77494'}

{'brewery_city': 'Tomball',
 'major_city': 'Houston',
 'state': 'TX',
 'zipcode': '77070'}

{'brewery_city': 'Galveston',
 'major_city': 'Galveston',
 'state': 'TX',
 'zipcode': '77550'}

{'brewery_city': 'Pharr',
 'major_city': 'Pharr',
 'state': 'TX',
 'zipcode': '78577'}

IndexError: list index out of range

In [60]:
all_breweries

[{'Name': '111 Brewing',
  'Street Address': 111,
  'Road': 'S FANNIN AVENUE',
  'City': 'DENISON',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '4/10/2019',
  'Status': 'In the works',
  'untappd': [{'brewery': {'brewery_id': 66888,
     'beer_count': 65,
     'brewery_name': 'Milepost 111 Brewing Company',
     'brewery_slug': 'milepost-111-brewing-company',
     'brewery_page_url': '/milepostbeer',
     'brewery_label': 'https://untappd.akamaized.net/site/brewery_logos/brewery-Milepost111_66888_590e6.jpeg',
     'country_name': 'United States',
     'location': {'brewery_city': 'Cashmere',
      'brewery_state': 'WA',
      'lat': 47.5244,
      'lng': -120.47}}}]},
 {'Name': '2 Docs Brewing Co',
  'Street Address': 502,
  'Road': 'TEXAS AVENUE',
  'City': 'LUBBOCK',
  'State': 'TX',
  'Brewery License': 'Brewery',
  'License Date': '10/5/2018',
  'Status': 'Open',
  'untappd': [{'brewery': {'brewery_id': 420745,
     'beer_count': 22,
     'brewery_name': 'Two